In [1]:
from __future__ import print_function
import numpy as np
from lmfit import Parameters, minimize
import matplotlib.pyplot as plt
from optimization import *
from export import *
import time
import sys


def boolean_translate(dct, key):
    try:
        answ = str(dct[key])
    except KeyError:
        return True
    if answ.lower() == 'false':
        answ = False
    elif answ.lower() == 'true':
        answ = True
    else:
        raise ValueError('Incorrect %s value for str_param'%key)
    
    return answ    


def onegaussian_fit(num_param, str_param, project_name='onegauss', graph=True, table=True, datfile=True):

    try:
        project_name = str_param['project_name']
    except KeyError:
        pass
    # will look like onegauss_2016-8-31-12-58-48
    prefix = project_name + '_' + '-'.join([str(i) for i in time.gmtime()[:6]])
    os.mkdir(prefix)
    os.chdir(prefix)

    graph_print = boolean_translate(str_param, 'graph_print')
    table_print = boolean_translate(str_param, 'table_print')
    normalize = boolean_translate(str_param, 'normalize')

    table = str_param['table']
    xserver = str_param['xserver']
    table = get_grd(table, xserver=xserver)
    
    bragg = num_param['bragg'].value
    template = str_param['template']
    real_data = str_param['data']
    if template == 'xy':
        theta, yelid = get_dat(real_data, bragg=bragg, normalize=normalize, template=template)
        yelid_errors = None
    elif template == 'xyy' or template == 'xyyerror':
        theta, yelid, yelid_errors = get_dat(real_data, bragg=bragg, normalize=normalize, template=template)
    elif template == 'xxerroryyerror':
        theta, theta_error, yelid, yelid_errors = get_dat(real_data, bragg=bragg, normalize=normalize, template=template)
    
    # now we have yelid with correct angles (not relative to bragg), 
    # errors and standing wave table as np.array() with correct order
    
    out = minimize(residual_onegaussian, num_param, args=(table, theta, yelid, yelid_errors))
    
    model = intensity_onegaussian(table,
                                  theta,
                                  out.params['amp'],
                                  out.params['sigma'],
                                  out.params['x0'],
                                  out.params['zmax'],
                                  out.params['angle_slope'],
                                  out.params['zmin']
                                  )
    
    chisquared = residual_onegaussian(out, table, angles, yelid, errors=yelid_errors)
    rfactor = sum(abs(model - data)) / sum(data)
    
    return out, angles, model, chisquared, rfactor

num_param, str_param = get_initials('/home/errorochka/Dropbox/DESY/pyXSW/pyxsw.prm')
params, theta, model, chisquared, rfactor = onegaussian_fit(num_param, str_param, project_name='test')

print('Chisquared = %.4f, rfactor = %.4f'%(chisquared, rfactor))
plt.plot(theta, model)
plt.show()

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


KeyError: 'zmin'

In [2]:
from __future__ import print_function
import numpy as np
from lmfit import Parameters, minimize
import matplotlib.pyplot as plt
from optimization import *
from export import *
import time
import sys

In [29]:
a = {1:'False', 2:'false', 3:'TruE', 4:'true', 5:120}
for key in a.keys():
    print(key, a[key], boolean_translate(a, key), sep='\t')

1	False	False
2	false	False
3	TruE	True
4	true	True


ValueError: Incorrect 5 value for str_param